# Manage users

In this scenario, you are going to learn how d365fo.integrations can be leveraged to manage users in Dynamics 365 for Finance and Operations.
The notebook will walk you through retrieving, creating, updating and deleting users.
Along the way, you will learn concepts of OData entities and how to interact with them using d365fo.integrations.

## Prerequisites

This notebook assumes that d365fo.integrations is installed and you have a basic understanding of it, how to configure it for use and how to use `Get-Help` and `Get-Command` to discover available commands and their usage. If you are new to d365fo.integrations or want a refresher, please refer to the [Getting Started](./getting_started.ipynb) notebook.

Use the following cell to create a temporary configuration. If you already have a configuration, you can skip this step.

In [ ]:
$tenantId = Read-Host -Prompt "Enter your tenant id"
$clientId = Read-Host -Prompt "Enter your client id"
$clientSecret = Read-Host -Prompt "Enter your client secret"
$name = Read-Host -Prompt "Enter a name for the configuration"
$url = Read-Host -Prompt "Enter the URL of the D365FO environment"

Add-D365ODataConfig -Tenant $tenantId -ClientId $clientId -ClientSecret $clientSecret -Name $name -Url $url -Temporary
Set-D365ActiveODataConfig -Name $name -Temporary

# The SystemUser entity

For managing users, we will be using the `SystemUsers` OData entity. To make it easier to work with this entity and allow you to change it to another entity, the next cell stores the entity name in a variable.

In [31]:
$entityName = "SystemUsers"

## Plural or singular name

Note that the actual entity name is `SystemUser` (singular) and not `SystemUsers` (plural). OData has a concept **EntitySetName** which is the name of the entity set exposed by the OData service. The entity set name is usually the plural form of the entity name. In this case, the entity set name is `SystemUsers`. In most situations, OData will want you to use the entity set name instead of the entity name. 

In practice it can get confusing when to use the singular and when to use the plural name. Some cmdlets of d365fo.integrations help you by accepting both the singular and plural form of the entity name. It will then determine the right form to use behind the scenes when making the request to the OData service.

## Entity metadata

Another important concept to know when working with OData entities is the entity metadata. The metadata contains information about the entity, its properties, navigation properties, and other information. 

To retrieve the metadata for an entity, you can use the `Get-D365ODataPublicEntity` cmdlet. You can run the cmdlet without any parameters, which will retrieve the metadata for all entities. It is not advisable to do that, since there are thousands of entities. Instead, use the `-EntityName`, `-EntityNameContains`, or `-ODataQuery` parameters to retrieve the entities you are interested in. For the purpose of this notebook, we will use the `-EntityName` parameter to retrieve the metadata for the `SystemUser` entity.

Note that in the following cell, the result of the `Get-D365ODataPublicEntity` cmdlet is stored in the `$entityMetadata` variable. This is because the cmdlet returns the metadata as a custom object, which can be used to explore the metadata further. The next line is just the variable, which will display the metadata stored in it.

In [32]:
$entityMetadata = Get-D365ODataPublicEntity -EntityName $entityName
$entityMetadata


Name                 : SystemUser
EntitySetName        : SystemUsers
LabelId              : @SYS12108
IsReadOnly           : False
ConfigurationEnabled : True
Properties           : {@{Name=UserID; TypeName=Edm.String; DataType=String; 
                       LabelId=@Kernel:UserInfo_ID; IsKey=True; IsMandatory=True; 
                       ConfigurationEnabled=True; AllowEdit=False; AllowEditOnCreate=True; 
                       IsDimension=False; DimensionRelation=; IsDynamicDimension=False; 
                       DimensionLegalEntityProperty=; DimensionTypeProperty=}, 
                       @{Name=WorkflowLineItemNotificationFormat; TypeName=Microsoft.Dynamics.DataE
                       ntities.WorkflowLineItemNotificationFormat; DataType=Enum; 
                       LabelId=@SYS332736; IsKey=False; IsMandatory=False; 
                       ConfigurationEnabled=True; AllowEdit=True; AllowEditOnCreate=True; 
                       IsDimension=False; DimensionRelation=; IsDyna

## Exploring the metadata

If the display of the metadata feels a bit overwhelming, don't worry. Other notebooks will explore the metadata in more detail. For now, we are interested in the `Properties` item of the metadata, which is a list of the fields of the entity. To view it in a more readable manner, run the next cell.

In [33]:
$entityMetadata.Properties | 
  Select-Object Name, DataType, IsKey, IsMandatory | 
  Sort-Object IsMandatory, IsKey, Name -Descending | 
  Format-Table -AutoSize


Name                                             DataType IsKey IsMandatory
----                                             -------- ----- -----------
UserID                                           String    True        True
UserName                                         String   False        True
UserInfo_language                                String   False        True
NetworkDomain                                    String   False        True
Helplanguage                                     String   False        True
WorkflowLineItemNotificationFormat               Enum     False       False
UserInfo_defaultPartition                        Enum     False       False
Theme                                            Enum     False       False
StartPage                                        String   False       False
SqmGUID                                          Guid     False       False
SqmEnabled                                       Enum     False       False
ShowNotific

If the PowerShell code is not clear to you, here is a quick breakdown:
- The `$entityMetadata.Properties` part retrieves the `Properties` item from the metadata.
- The `Properties` item is a list of fields of the entity where each field has its own metadata.
- The `Select-Object` cmdlet is used to select the `Name`, `DataType`, `IsKey`, and `IsMandatory` items from the field metadata.
- The `Sort-Object` cmdlet is used to sort the fields by the `IsMandatory`, `IsKey`, and `Name` metadata.
- The `Format-Table` cmdlet is used to display the fields in a table format.

> ℹ️ **Note** It is beyond the scope of this notebook to explain all those cmdlets and how they are connected in more detail. This is part of the base functionality of PowerShell and not specific to d365fo.integrations. If you want to learn more about PowerShell in general, there are many resources available online. One good starting point is [PowerShell 101](https://learn.microsoft.com/en-us/powershell/scripting/learn/ps101/00-introduction).

This gives us a nice overview of the fields of the `SystemUser` entity. At the top are the fields that are mandatory when creating a new user. The `UserID` field where the `IsKey` metadata item is `True` will become important when updating or deleting a user.

# Retrieve users

That's enough theory for now. Let's start by retrieving all users in Dynamics 365 for Finance and Operations. To do this, we will use the `Get-D365ODataEntityData` cmdlet. The cmdlet requires the `-EntityName` parameter, which we will set to `SystemUsers`. This is not quite correct (remember the paragraph about singular and plural names), but the cmdlet will handle this for us. If you want to be more explicit, you can use the `-EntitySetName` parameter and set it to `SystemUsers`.

We also use the `-Top 5` parameter to limit the number of users retrieved. While you are testing, it is a good idea to limit the number of records retrieved to avoid long wait times and to not overload the system.

Again, we use the `Select-Object` cmdlet to select the `UserID`, `UserName`, and `Enabled` fields which we are interested in. Try changing the fields if you are interested in other fields.

In [34]:
Get-D365ODataEntityData -EntityName $entityName -Top 5 |
  Select-Object UserID, UserName, Enabled |
  Format-Table -AutoSize


UserID               UserName             Enabled
------               --------             -------
Admin                Admin                   True
FRServiceUser        FRServiceUser           True
SysHealthServiceUser SysHealthServiceUser    True
ScaleUnitManagement  ScaleUnitManagement     True
PowerPlatformApp     PowerPlatformApp        True



> ⚠️ **Warning**: Microsoft users
>
> You may notice some users with ids and names that are unfamiliar to you (for example a *SysHealthServiceUser*). These are system users added by Microsoft and are used for various purposes. You will not see them in the user interface of Dynamics 365 for Finance and Operations, as they are filtered by the `isMicrosoftAccount` field. Unfortunately, this filter is not applied when retrieving users via OData. There is also no field in the entity that can be used to filter out these users. So be aware of this when making changes to users. These Microsoft users should not be changed.

# Updating users

To update a user, we need to get familiar with the `Update-D365ODataEntity` cmdlet. This, like previous cmdlets, has a `-EntityName` parameter which we will set to `SystemUsers`. 

## Entity key

Now updates require a single record to be updated. To ensure this, OData requires that the values of the key fields of an entity are provided. Remember the `IsKey` metadata? Only the `UserID` field has that metadata set to `True`. 

Another way to find out the key fields of an entity is to use the `Get-D365ODataEntityKey` cmdlet. The following cell shows how to use it.

In [35]:
$entityKey = Get-D365OdataEntityKey -Name $entityName -Properties $entityMetadata.Properties
$entityKey


Name        Keys
----        ----
SystemUsers @{FieldName=UserID; DataType=String}



Another easier option is to "pipe" the full metadata object to the `Get-D365ODataEntityKey` cmdlet and let it sort out itself what it needs from it to determine the key. This is shown in the next cell. 

> ℹ️ **Note** "Piping" is another concept of PowerShell where the output of one cmdlet is passed as input to another cmdlet. It is done by using the `|` character.

In [36]:
$entityKey = $entityMetadata | Get-D365OdataEntityKey
$entityKey


Name       Keys
----       ----
SystemUser @{FieldName=UserID; DataType=String}



Now that we know the key field of the entity, we need the value for it. In this case, the value is a user id. Run the next cell to provide a user id of your choice, which will be stored in the `$userId` variable.

In [ ]:
$userId = Read-Host -Prompt "Enter the user id"

With the user id value, we can now set the `-Key` parameter of the `Update-D365ODataEntity` cmdlet. It expects a string in the format `KeyField1='Value1',KeyField2='Value2',...`. In our case, it will be `-Key "UserID='$userId'"`.


## Payload

Now that we know the key of the entity, the only thing missing is the payload. It determines which fields will be updated and their new values. The payload is provided in the [JSON](https://www.json.org/) (JavaScript Object Notation) format. The JSON format is a lightweight data interchange format that is easy for humans to read and write and easy for machines to parse and generate.

In this case, we will update the `Enabled` field of the user. The `Enabled` field is a boolean field, which means it can have two values: `True` or `False`. We will set it to `False`. The JSON format for this is `{"Enabled": "False"}`. Let's store that in a variable using the next cell.

In [37]:
$payloadJSON = '{"Enabled": "False"}'

## Disable a user

And now we have everything we need to update a user to disable it. The next cell will first display all the values of the variables we will be using:
- `$entityName` - the entity name
- `$userId` - the user id
- `$payloadJSON` - the payload in JSON format

It will then call the `Update-D365ODataEntity` cmdlet to update the user. If everything goes well, the cmdlet will disable the user.

In [ ]:
$entityName
$userId
$payloadJSON

Update-D365ODataEntity -EntityName $entityName -Key "UserID='$userId'" -Payload $payloadJSON

Now try changing the `$payloadJSON` variable in the cell of the [Payload](#payload) section so that when the previous cell is executed once more, the user will be enabled again.

If you want to update more than one field, the payload can get more complex. We will see in the next part on user creation how such payloads can be created.

# Creating users

Creating data (in this case users) is usually more complex than updating it, since it requires to provide values for all mandatory fields and usually also additional fields. Which fields exactly need to be provided and what their values should be is specific to each entity and the domain and business logic behind it. The metadata of an entity is often not enough to determine this, as you will see in this case. So be prepared to spend some time on figuring out how to create data in other entities.

To do the data creation, we will use the `Import-D365ODataEntity` cmdlet, which, like the `Update-D365ODataEntity` cmdlet, requires a payload in the JSON format.

## Prepare the payload

From the [Exploring the metadata](#exploring-the-metadata) section, we know the five mandatory fields of the entity:
- `UserID`: the unique identifier of the new user
- `UserName`: the new user's name
- `UserInfo_language`: the language the user will see the application in
- `NetworkDomain`: a url that will be used in the authentification of the user
- `Helplanguage`: the alternate help language

### NetworkDomain

Most of the mandatory fields should be easy to fill out, but the `NetworkDomain` field might be a bit tricky. In the user interface, it appears with the name "Provider" and usually contains the URL "https://sts.windows.net/". This URL is used for the authentication of the user. For users in your tenant, you can use the same URL. For other users, their tenant might need to be added to the URL or a different URL might be needed. This is out of the scope of this notebook, but be aware of this when creating users.

With that information, let's create the payload for a new user. The next cell will set and query you for the values of the mandatory fields and store them in variables.

In [4]:
$userId = Read-Host -Prompt "Enter the user id"
$userName = Read-Host -Prompt "Enter the user name"
$userLanguage = "en-us" # Change this to the language of your users or replace it with a Read-Host to provide a value of your choice
$networkDomain = "https://sts.windows.net/" # Change this according to your needs

Then let's create the payload. Since there are multiple fields, we will first declare a PowerShell object with the names and values of the fields. We can then use the PowerShell `ConvertTo-Json` cmdlet to convert the object to a JSON string. This is easier than declaring the JSON string directly.

Also note we are using the `$userLanguage` for both the `UserInfo_language` and `Helplanguage` fields. These fields expect a value as in the "Language/locale" column of [Language and locale descriptors in Help](https://learn.microsoft.com/en-us/dynamics365/fin-ops-core/dev-itpro/help/language-locale).

In [ ]:
$payloadObject = @{
  UserID = $userId
  UserName = $userName
  UserInfo_language = $userLanguage
  NetworkDomain = $networkDomain
  Helplanguage = $userLanguage
}

$payloadJSON = $payloadObject | ConvertTo-Json
$payloadJSON

Do you think that payload is sufficient to create a user? Let's find out.

In [6]:
$createdEntity = Import-D365ODataEntity -EntityName $entityName -Payload $payloadJSON

[13:40:52][Import-D365ODataEntity] Something went wrong while importing data through the OData endpoint for the entity: SystemUsers | Response status code does not indicate success: 400 (Bad Request).


Error: Command failed: SubmitCode: $createdEntity = Import-D365ODataEntity -EntityNam ...

## Analyze errors

If you followed the notebook so far, you will get an error when trying to create a user with that payload. The error message will be something like 

>Something went wrong while importing data through the OData endpoint for the entity: SystemUsers | Response status code does not indicate success: 400 (Bad Request).

This is not very helpful, but it is a common error message when the payload is not correct. So how can you go about figuring out what is wrong with the payload?

To do that, we will use a couple of PowerShell features. The first is one of the [common parameters](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_commonparameters) of PowerShell. These parameters are available on all cmdlets. The one we are interested in is the `-Verbose` parameter. When this parameter is set, the cmdlet will output more information about what it is doing. This can be very helpful when debugging issues. Let's run the `Import-D365ODataEntity` again with that parameter. Note that we do not supply a value to the parameter. It is a so-called switch parameter, which means that it is either present or not. If it is present, it is considered to be `$true`.

In [ ]:
$createdEntity = Import-D365ODataEntity -EntityName $entityName -Payload $payloadJSON -Verbose

This gives us a few more lines of VERBOSE output, but nothing really helpful. At least it looks like the cmdlet was able to send the request to the OData service and received an answer. But you may already have guessed that from the "400 (Bad Request)" part of the previous error.

Ok, so `-Verbose` does not help here. Still, it is good to keep in mind, as it is an easy and fast way to get more information about what a cmdlet is doing.

The next feature we will use is the `$error` variable. This variable is maintained by PowerShell in the background. It contains all errors that occurred during the execution of the script. You can access the last error by using `$error[0]`. This is very helpful when you want to know what went wrong in the last command. Let's see what the last error was.

In [7]:
$error[0]

Stop-PSFFunction: Something went wrong while importing data through the OData endpoint for the entity: SystemUsers



We already know that error. Let's try the one before that. By incrementing the number in the square brackets, you can go back in time and see previous errors.

In [12]:
$error[1]

Invoke-RestMethod:  {   "error": {     "code": "",     "message": "An error has occurred.",     "innererror": {      
"message": "Write failed for table row of type \u0027SystemUserEntity\u0027. Infolog: Error: Object
reference not set to an instance of an object..",       "type":
"Microsoft.Dynamics.Platform.Integration.Services.OData.AxODataWriteException",       "stacktrace":
"   at
Microsoft.Dynamics.Platform.Integration.Services.OData.Update.UpdateProcessor.CreateEntity_Save(Chan
geOperationContext context, ChangeInfo changeInfo)\r\n   at
Microsoft.Dynamics.Platform.Integration.Services.OData.Update.ChangeInfo.ExecuteActionsInCompanyCont
ext(IEnumerable\u00601 actionList, ChangeOperationContext operationContext)\r\n   at
Microsoft.Dynamics.Platform.Integration.Services.OData.Update.ChangeInfo.TrySave(ChangeOperationCont
ext operationContext)\r\n   at
Microsoft.Dynamics.Platform.Integration.Services.OData.Update.UpdateManager.SaveChanges()\r\n   at
Microsoft.Dynamics.Platform.Integ

Unfortunately, while we can see from the stack trace that the error occurred in the Microsoft.Dynamics.Platform.Integration.Services.OData assembly, we still do not know what is wrong with the payload. The error message is not very helpful either. So what can we do now?

The next steps in the analysis would include research on whether others encountered and solved this issue. It can also include using the entity in different ways, e.g. through the Excel AddIn or the Data Management workspace. However, these steps are out of scope for this notebook. 

While the error analysis did not provide a solution, it gives you an idea of how to approach such issues and what is involved in getting a Dynamics 365 Finance and Operations integration working. For the purpose of this notebook, we assume the analysis was done and provided us with a working payload.

## Create a user

To create a working payload, we need to provide values for two more fields:
- `Alias`: despite the name, this field holds the user's email address; this is required to validate the user against the Azure Entra ID
- `AccountType`: this needs to be "ClaimsUser"

> ℹ️ **Note** The entity also has an "Email" field, which is not required. It is used when the system sends emails to or on behalf of the user.

The following cell requests the email address from you and adds the values to the existing `$payloadObject`. It then again converts it into a JSON string.

In [ ]:
$emailAddress = Read-Host -Prompt "Enter the email address of the user"
$accountType = "ClaimsUser"

$payloadObject.Alias = $emailAddress
$payloadObject.AccountType = $accountType

$payloadJSON = $payloadObject | ConvertTo-Json
$payloadJSON

And now we can finally run the `Import-D365ODataEntity` cmdlet and hopefully get a successfully created user.

In [ ]:
$createdEntity = Import-D365ODataEntity -EntityName $entityName -Payload $payloadJSON
$createdEntity

You may notice that the user that was created is not enabled. This is because the `Enabled` field is not mandatory and is set to `False` by default. If you want to enable the user, you would need to add the `Enabled` field to the payload and set it to `True`. You can also set other fields such as the `Company` so that the user starts their session in the right legal entity. Feel free to experiment with the previous cells and the payload or create new cells to do so.

# Deleting users

For the last part of this scenario notebook, we will look at deleting a record in an entity. This is done with the `Remove-D365ODataEntity` cmdlet. Like all previous cmdlets, it requires the `-EntityName` parameter, which we will set to `SystemUsers`. Like the `Update-D365ODataEntity` cmdlet, it requires the key of the entity to be provided. We already know the key of the `SystemUser` entity, which is the `UserID` field. We will use the same user id as before to delete the user.

That is already all you ned to know. Look at the next cell to see how the user is deleted.

In [30]:
Remove-D365ODataEntity -EntityName $entityName -Key "UserID='$userId'"

# Conclusion

Congratulations on making it through this notebook. You have learned how to retrieve, create, update, and delete users in Dynamics 365 for Finance and Operations. You have also learned about OData entities, their metadata, and how to interact with them using d365fo.integrations. This is a great foundation for further exploration of the capabilities of d365fo.integrations and Dynamics 365 for Finance and Operations.